In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()
import pandas as pd

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)
df.show()

+--------------------+-------------------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|               date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+-------------------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08 00:00:00|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13 00:00:00|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12 00:00:00|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16 00:00:00|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08 00:00:00|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa0052

In [4]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')

In [5]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("""SELECT YEAR(date) AS year, ROUND(AVG(price), 2) AS average_price
FROM home_sales
WHERE bedrooms = '4'
GROUP BY YEAR(date)
ORDER BY YEAR(date)""").show(10)


+----+-------------+
|year|average_price|
+----+-------------+
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
|2022|    296363.88|
+----+-------------+



In [6]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("""SELECT date_built, ROUND(AVG(price), 2) AS average_price
FROM home_sales
WHERE bedrooms = '3'AND
bathrooms = '3'
GROUP BY date_built
ORDER BY date_built""").show(10)


+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [7]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("""SELECT date_built, ROUND(AVG(price), 2) AS average_price
FROM home_sales
WHERE bedrooms = '3'AND
bathrooms = '3' AND
floors = '2' AND
sqft_living >= '2000'
GROUP BY date_built
ORDER BY date_built""").show(10)


+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|     293965.1|
|      2017|    280317.58|
+----------+-------------+



In [8]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()
spark.sql("""SELECT date_built, ROUND(AVG(view), 2) AS average_view_rating
FROM home_sales
WHERE price >= '350000'
GROUP BY date_built
ORDER BY date_built
""").show(10)

print("--- %s seconds ---" % round((time.time() - start_time), 2))

+----------+-------------------+
|date_built|average_view_rating|
+----------+-------------------+
|      2010|              30.64|
|      2011|              32.28|
|      2012|              33.48|
|      2013|              32.61|
|      2014|              32.28|
|      2015|              31.89|
|      2016|              32.91|
|      2017|              32.03|
+----------+-------------------+

--- 0.8 seconds ---


In [9]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [10]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [11]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()
spark.sql("""SELECT date_built, ROUND(AVG(view), 2) AS average_view_rating
FROM home_sales
WHERE price >= '350000'
GROUP BY date_built
ORDER BY date_built
""").show(10)
print("--- %s seconds ---" % round((time.time() - start_time), 2))


+----------+-------------------+
|date_built|average_view_rating|
+----------+-------------------+
|      2010|              30.64|
|      2011|              32.28|
|      2012|              33.48|
|      2013|              32.61|
|      2014|              32.28|
|      2015|              31.89|
|      2016|              32.91|
|      2017|              32.03|
+----------+-------------------+

--- 0.65 seconds ---


Unfortunately I was not able to resolve the following error and create a formatted parquet home sales data and could not get the results but I still have completed the code for the rest of the challenge. 

In [12]:
df.write.parquet('parquet_home_sales', mode='overwrite')

Py4JJavaError: An error occurred while calling o45.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:651)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:288)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:98)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$6(SQLExecution.scala:109)

In [32]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
partitioned = df.write.partitionBy("date_built").mode("overwrite").parquet('parquet_home_sales')

In [ ]:
# 12. Create a temporary table for the parquet data.
partitioned.createOrReplaceTempView('partitioned_home_sales')

In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

start_time = time.time()
spark.sql("""SELECT date_built, ROUND(AVG(view), 2) AS average_view_rating
FROM home_sales
WHERE price >= '350000'
GROUP BY date_built
ORDER BY date_built
""").show(10)
print("--- %s seconds ---" % (time.time() - start_time))

In [33]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [34]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached('home_sales')

False